In [10]:
# Tiffany Yu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipywidgets import interact

In [11]:
import sys
!{sys.executable} -m jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [12]:
def create_dataframe(countryList, cont_var, yearList):
    # Read data
    df=pd.read_csv("./data/data-raw.csv")

    # Rename market -> country
    df = df.rename({'market':'country'}, axis=1)
    # Rename quarters to numbers
    df['quarter'].mask(df['quarter'] == 'January-March', 1, inplace=True)
    df['quarter'].mask(df['quarter'] == 'April-June', 2, inplace=True)
    df['quarter'].mask(df['quarter'] == 'July-September', 3, inplace=True)
    df['quarter'].mask(df['quarter'] == 'October-December', 4, inplace=True)
    # Change string of years to int
    df["year"] = df['year'].apply(lambda x: int(x) if x.isnumeric() else 2020)

    # # Filter dataframe by year, country
    df = df[df["country"].isin(countryList)]
    df = df.loc[(df["year"]>=min(yearList)) & (df["year"]<=max(yearList))]

    # Group dataframe by country, quarter
    df = df[["quarter","country",cont_var]].groupby(["country","quarter"], as_index=False).sum()

    return df


In [13]:
def plot_bars_and_line(countryList, cont_var, yearList):
    # Get data
    df = create_dataframe(countryList, cont_var, yearList)

    # Variables for grouped bar plot
    width = 1/(len(countryList)+1)
    labels = ['Q1', 'Q2', 'Q3', 'Q4']
    x_pos_ref = np.arange(len(labels))
    
    # Create axis
    ax = plt.gca()
    
    # Plot each country's data as a bar graph
    for i,country in enumerate(countryList):
        # Filter by country
        df_country = df.loc[df["country"]==country]
        # Get y axis points for plotting
        df_country_var = df_country[cont_var].values
        # Plot
        ax.bar(x_pos_ref+(i*width)-1, df_country_var, width, label=country)

    # Format first plot
    x_pos_tick = x_pos_ref-1+(0.5*width*(len(countryList)-1))
    ax.set_ylabel(cont_var+" (bar plot)")
    ax.set_xticks(x_pos_tick)
    ax.set_xticklabels(labels)
    ax.legend(bbox_to_anchor=(1.2, 1), loc='upper left')
    
    # Create second axis
    ax2=ax.twinx()
    
    # Calculate sums for each quarter, get y values for plotting line
    df_annual_sum = df.groupby(["quarter"], as_index=False).sum()
    y_annual_sum = list(df_annual_sum[cont_var].values)
    
    # Plot line graph for quarter totals
    ax2.plot(x_pos_tick, y_annual_sum, marker='o', color='r')
    
    # Format second plot
    ax2.set_ylabel(
        "Total "+cont_var.lower()+" (line plot)", 
        rotation=270,
        labelpad=20,
        color='r',
    )
    ax2.set_ylim(bottom=0, top=max(y_annual_sum)*1.1)
    
    # Format figure
    plt.title(cont_var+' by quarter from '+str(min(yearList))+"-"+str(max(yearList)))


In [14]:
# Get list of countries
df=pd.read_csv("./data/data-raw.csv")
countries_all = sorted(list(set(df["market"].values)))

# Create interactive plot
interact(
    plot_bars_and_line,
    yearList = widgets.IntRangeSlider(
        description='Year',
        value=[2002, 2020],
        min=2002,
        max=2020,
        step=1,
        orientation='horizontal',
    ),
    countryList = widgets.SelectMultiple(
        description='Country',
        options=countries_all,
        value=["Argentina","Austria","Belgium"],
    ),
    cont_var = widgets.Dropdown(
        description="Data type",
        options=["Visits (000s)","Spend (£m)","Nights (000s)"],
        value="Visits (000s)"
    )
)

interactive(children=(SelectMultiple(description='Country', index=(0, 2, 4), options=('Argentina', 'Australia'…

<function __main__.plot_bars_and_line(countryList, cont_var, yearList)>